In [1]:
import os
from itertools import product
from collections import namedtuple
import time
from typing import Tuple
import random

from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches
import numpy as np
import pandas as pd

# Подключаем локальные пакеты
from pyqumo.random import Distribution, Exponential, HyperExponential, Erlang
from pyqumo.cqumo.sim import simulate_tandem


from pyqumo.fitting.johnson89 import fit_mern2
from pyqumo.stats import get_cv, get_skewness, get_noncentral_m2, get_noncentral_m3
from pyqumo.random import HyperErlang

# Поключаем пакеты для ML
# import math
# from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, f1_score
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.neural_network import MLPClassifier
# from sklearn.linear_model import LinearRegression, LogisticRegression
# from catboost import CatBoostClassifier
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow import keras
# from tensorflow.python.keras import engine
# from tensorflow.keras import layers
# import seaborn as sns
# from scipy import stats

In [2]:
def fit_ph(avg: float, std: float) -> Distribution:
    """
    Возвращает PH-распределение по стандартному отклонению и среднему значению.
    """
    cv = std / avg
    if (cv == 1) or np.isclose(avg, std, rtol=0.1):
        return Exponential(avg)
    if cv > 1:
        return HyperExponential.fit(avg, std)
    return Erlang.fit(avg, std)

In [3]:
 COLUMNS = {'sta_where_packets_arrive',
            'simulation_type',
            'num_generated_packets',
            'mode',
            'num_stations',
            'queue_capacity',
            'mean_arrival_time',
            'std_arrival_time',
            'bitrate',
            'mean_packet_size',
            'std_mean_packet_size'}

In [4]:
# def simulate(
#     df: pd.DataFrame, *,
#     arr_avg: float,
#     arr_cv: float,
#     arr_skewness: float,
#     srv_avg: float,
#     srv_cv: float,
#     srv_skewness: float,
#     net_size: int,
#     capacity: int,
#     num_packets: int,
#     force: bool = False
# ) -> pd.DataFrame:

#     # row_df = df[
#     #     (df.ArrAvg == arr_m1) &
#     #     (df.ArrCV == arr_m2) &
#     #     (df.ArrM3 == arr_m3) &
#     #     (df.ArrAvg == arr_m1) &
#     #     (df.ArrM2 == arr_m2) &
#     #     (df.ArrM3 == arr_m3) &
#     #     (df.SrvM1 == srv_m1) &
#     #     (df.SrvM2 == srv_m2) &
#     #     (df.SrvM3 == srv_m3) &
#     #     (df.NetSize == net_size) &
#     #     (df.Capacity == capacity)]
    
#     # # Вычислим признаки, которые говорят о необходимости пересчета:
#     # no_row = len(row_df) == 0
#     # not_enough_packets = (not no_row) and (row_df.NumPackets.iloc[0] < num_packets)

#     arr_m1 = arr_avg
#     arr_m2 = get_noncentral_m2(arr_m1, arr_cv)
#     arr_m3 = get_noncentral_m3(arr_m1, arr_cv, arr_skewness)

#     srv_m1 = srv_avg
#     srv_m2 = get_noncentral_m2(srv_m1, srv_cv)
#     srv_m3 = get_noncentral_m3(srv_m1, srv_cv, srv_skewness)


#     # # Проверим, нужно ли пересчитать результаты:
#     # if force or no_row or not_enough_packets:
#     arr,_  = fit_mern2(moments=[arr_m1, arr_m2, arr_m3], strict=False)
#     srv,_ = fit_mern2(moments=[srv_m1, srv_m2, srv_m3], strict=False)
#     ret = simulate_tandem(arr, [srv] * net_size, capacity, num_packets)

#     row_data = {
#         'ArrM1': arr_m1,
#         'ArrM2': arr_m2,
#         'ArrM3': arr_m3,
#         'ArrAvg': arr_m1,
#         'ArrStd': get_std(arr_m1, arr_m2),
#         'ArrCv': arr_cv,
#         'ArrSkewness': arr_skewness,
#         'SrvM1': srv_m1,
#         'SrvM2': srv_m2,
#         'SrvM3': srv_m3,
#         'SrvAvg': srv_m1,
#         'SrvStd': get_std(srv_m1, srv_m2),
#         'SrvCv': srv_cv,
#         'SrvSkewness': srv_skewness,
#         'Rho': arr_m1 / srv_m1,
#         'NetSize': net_size,
#         'Capacity': capacity,
#         'NumPackets': num_packets,
#         'DelayAvg': ret.delivery_delays[0].avg,
#         'DelayStd': ret.delivery_delays[0].std,
#         'DeliveryProb': ret.delivery_prob[0],
#     }
#     # # Если строки еще вообще не было, добавляем ее, а если была - обновляем:
#     # if no_row:
#     #     df = df.append(row_data, ignore_index=True)
#     # else:
#     #     df.update(pd.DataFrame(row_data, index=[row_df.index[0]]))
#     df = df.append(row_data, ignore_index=True)

#     return df

In [5]:
dir(simulate_tandem)
print(simulate_tandem.__doc__)

simulate_tandem(arrivals, services, queue_capacity: int, bool fixed_service: bool = False, max_packets: int = 100000) -> TandemResults

    Simulate tandem using C++ implementation. No cross-traffic supported.

    Example
    -------
    >>> simulate_tandem(
    >>>     Erlang(2, 1),
    >>>     [Exponential(5), Exponential(6), Exponential(3)],
    >>>     10,  # queue capacity
    >>>     1000000)  # number of packets (1 million)

    To simulate a network, where cross-traffic arrives at stations 0, 2 and 3,
    and network has length 5:

    >>> simulate_tandem(
    >>>     [Erlang(2, 1), None, Exponential(2), Exponential(2.3), None],
    >>>     [Exponential(3) for _ in range(5)],
    >>>     10,  # queue capacity
    >>>     1000000)  # number of packets (1 million)

    Results returned are defined in pyqumo.sim.tandem.Results.

    Parameters
    ----------
    arrivals : Distribution instance, or list-like with optional Distributions
        if list-like, non-None values should

In [6]:
ret = simulate_tandem(Exponential(1), [Exponential(2)] * 3, 5, 10,)

TypeError: simulate_tandem() got multiple values for keyword argument 'fixed_service'

In [25]:
simulate_tandem()


AttributeError: 'builtin_function_or_method' object has no attribute 'tabulate'

In [23]:
print(ret.tabulate())

Param                        Value
---------------------------  ---------------------
Number of stations           3
Loss probability             [0.0, 0.0, 0.0]
[[ STATION #0 ]]
System size PMF              [0.81, 0.179, 0.0111]
System size average          0.20109202418258493
System size std.dev.         0.42772157368539415
Queue size PMF               [0.989, 0.0111]
Queue size average           0.011145861301587227
Queue size std.dev.          0.10498395628586783
Busy PMF                     [0.85, 0.15]
Utilization                  0.15016977434187417
Drop probability             0.0
Delivery probability         1.0
Departures, average          1.4979153192327428
Departures, std.dev.         1.6844102510868553
Response time, average       0.29986530147084994
Response time, std.dev.      0.17310226389114755
Wait time, average           0.019255650963801232
Wait time, std.dev.          0.05776695289140369
End-to-end delays, average   1.5186918574466535
End-to-end delays, std.dev.  0